## Use the full CDS for TFs, and promoter for genes
### Also, avoid using sequences in dataframe, use the names only. Keep sequences in a dict instead.

# Dataset control values

In [51]:
BP_considered = 2000 # base pairs upstream and downstream
FILTER_TF = True # only keep tfs that were marked as active in the data

## Read perturbation data

In [52]:
from tqdm import tqdm

In [53]:
import scanpy as sc

In [54]:
adata = sc.read_h5ad('RPE1_CRISPRa_mean_pop.h5ad')

In [55]:
adata

AnnData object with n_obs × n_vars = 10912 × 5424
    obs: 'protospacer', 'target_gene', 'target_expr', 'active', 'cluster', 'cluster_name', 'cluster_description', 'gene_driven', 'sequence_driven', 'sequence_suffix', 'bulk_expr', 'expressed', 'p_expressed', 'de_genes', 'strength', 'cell_count', 'target_gene_id', 'masked_active', 'masked_cluster', 'masked_cluster_name', 'masked_cluster_description', 'masked_cluster_size', 'masked_cluster_num_genes', 'stricter_masked_cluster', 'stricter_masked_cluster_description'
    var: 'feature_types', 'genome', 'mean', 'in_matrix', 'std', 'cv', 'fano', 'pairwise_p_cost', 'mean_adjusted_pairwise_p_cost', 'excess_cv', 'pairwise_chosen', 'gene_id'
    layers: 'adj_p', 'masked', 'p'

In [56]:
adata.obs

,protospacer,target_gene,target_expr,active,cluster,cluster_name,cluster_description,gene_driven,sequence_driven,sequence_suffix,...,cell_count,target_gene_id,masked_active,masked_cluster,masked_cluster_name,masked_cluster_description,masked_cluster_size,masked_cluster_num_genes,stricter_masked_cluster,stricter_masked_cluster_description
guide_identity,,,,,,,,,,,,,,,,,,,,,
AATF_GCGCAGAAGGTTGAAGGGAT,GCGCAGAAGGTTGAAGGGAT,AATF,2.898099,False,372.0,NaN,372: SEQUENCE-driven (GGGAT) (13),False,True,GGGAT,...,116.0,ENSG00000275700,False,1479.0,NaN,1479: SEQUENCE-driven (GGGAT) (13),13.0,13.0,2041.0,2041: SEQUENCE-driven (GGGAT) (7)
AATF_GCGTGCGAGTGCGCGGGAAG,GCGTGCGAGTGCGCGGGAAG,AATF,1.635568,False,918.0,NaN,singleton (GGAAG) (1),False,False,,...,53.0,ENSG00000275700,False,237.0,NaN,singleton (GGAAG) (1),1.0,1.0,323.0,singleton (GGAAG) (1)
AATF_GGAAGCGCGCAGAAGGTTGA,GGAAGCGCGCAGAAGGTTGA,AATF,2.614520,True,2187.0,AATF_1,2187: neither (6),False,False,,...,102.0,ENSG00000275700,False,1929.0,NaN,1929: neither (4),4.0,4.0,2685.0,2685: neither (4)
AATF_GGAAGGGATTGGAGCCGTAT,GGAAGGGATTGGAGCCGTAT,AATF,1.581901,True,2187.0,AATF_1,2187: neither (6),False,False,,...,108.0,ENSG00000275700,False,1094.0,NaN,1094: neither (6),6.0,6.0,1437.0,1437: neither (4)
AATF_GGGCGTTGCTAGCATGAAGG,GGGCGTTGCTAGCATGAAGG,AATF,2.612676,False,1112.0,NaN,singleton (GAAGG) (1),False,False,,...,67.0,ENSG00000275700,False,183.0,NaN,singleton (GAAGG) (1),1.0,1.0,265.0,singleton (GAAGG) (1)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
off-target_GGGAACAGGGGCGGTCCGTA,GGGAACAGGGGCGGTCCGTA,off-target,NaN,False,161.0,NaN,161: neither (45),False,False,,...,1721.0,NaN,False,1138.0,NaN,1138: SEQUENCE-driven (GTA) (19),19.0,19.0,1510.0,1510: SEQUENCE-driven (GTA) (13)
off-target_GTAAATTCTCGCGTAACGTT,GTAAATTCTCGCGTAACGTT,off-target,NaN,False,2142.0,NaN,2142: neither (7),False,False,,...,454.0,NaN,False,1131.0,NaN,1131: neither (7),7.0,7.0,1499.0,1499: neither (5)
off-target_GTCCATAGGGTCTAGCGCCG,GTCCATAGGGTCTAGCGCCG,off-target,NaN,False,395.0,NaN,395: SEQUENCE-driven (CCG) (48),False,True,CCG,...,813.0,NaN,False,1416.0,NaN,1416: SEQUENCE-driven (CCG) (30),30.0,30.0,1953.0,1953: SEQUENCE-driven (ACCG) (4)


In [57]:
if FILTER_TF:
    adata = adata[adata.obs["active"] == True]

In [58]:
adata.obs["target_gene"].value_counts()

target_gene
CCN3       6
DLX3       6
HNRNPAB    6
HELZ2      6
FOXF2      6
          ..
ZNF587     2
ZNF655     2
ZNF664     2
ZNF777     2
ZNF813     2
Name: count, Length: 223, dtype: int64

In [59]:
import pandas as pd
import numpy as np

In [60]:
adata.var

,feature_types,genome,mean,in_matrix,std,cv,fano,pairwise_p_cost,mean_adjusted_pairwise_p_cost,excess_cv,pairwise_chosen,gene_id
gene_name,,,,,,,,,,,,
DPM1,Gene Expression,GRCh38,0.497299,True,0.738486,1.484995,1.096648,NaN,NaN,0.977718,False,ENSG00000000419
CFH,Gene Expression,GRCh38,0.278384,True,0.554464,1.991723,1.104338,NaN,NaN,1.006758,False,ENSG00000000971
FUCA2,Gene Expression,GRCh38,0.905370,True,1.024018,1.131049,1.158215,-2.451047,0.066219,0.958416,False,ENSG00000001036
NIPAL3,Gene Expression,GRCh38,0.327920,True,0.624273,1.903735,1.188450,NaN,NaN,1.046037,False,ENSG00000001461
LAS1L,Gene Expression,GRCh38,0.263869,True,0.530470,2.010355,1.066434,NaN,NaN,0.996978,False,ENSG00000001497
...,...,...,...,...,...,...,...,...,...,...,...,...
AC010642.2,Gene Expression,GRCh38,0.609991,True,0.836759,1.371755,1.147828,-2.582480,0.019873,0.981516,False,ENSG00000283103
SCO2,Gene Expression,GRCh38,0.259892,True,0.526850,2.027185,1.068022,NaN,NaN,0.988185,False,ENSG00000284194
POLR2J3,Gene Expression,GRCh38,0.579630,True,0.874262,1.508309,1.318657,-2.655410,-0.042057,1.058342,False,ENSG00000285437


## Create dictionary of gene -> sequences

In [61]:
gff = pd.read_csv(
	"GRCh38_latest_genomic.gff",
	sep="\t",
	comment="#",
	header=None,
	names=["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"]
)

# Filter to only genes
genes = gff[gff["type"] == "gene"]
print(genes.shape)
print(genes.head())

(48106, 9)
           seqid      source  type  start    end score strand phase  \
19  NC_000001.11  BestRefSeq  gene  17369  17436     .      -     .   
28  NC_000001.11      Gnomon  gene  29774  35418     .      +     .   
33  NC_000001.11  BestRefSeq  gene  30366  30503     .      +     .   
38  NC_000001.11  BestRefSeq  gene  34611  36081     .      -     .   
45  NC_000001.11  BestRefSeq  gene  65419  71585     .      +     .   

                                           attributes  
19  ID=gene-MIR6859-1;Dbxref=GeneID:102466751,HGNC...  
28  ID=gene-MIR1302-2HG;Dbxref=GeneID:107985730,HG...  
33  ID=gene-MIR1302-2;Dbxref=GeneID:100302278,HGNC...  
38  ID=gene-FAM138A;Dbxref=GeneID:645520,HGNC:HGNC...  
45  ID=gene-OR4F5;Dbxref=GeneID:79501,HGNC:HGNC:14...  


## Read full genome

In [62]:
from pyfaidx import Fasta
from Bio.Seq import Seq

genome = Fasta("GRCh38_latest_genomic.fna")

In [63]:
gene_names = list(adata.var_names)

In [64]:
len(gene_names)

5424

In [65]:
assert len(gene_names) == len(set(gene_names)), "Gene names are not unique!"

In [66]:
def get_target_gene_sequence(gff_row):
	seqid = gff_row["seqid"]
	start = gff_row["start"]
	strand = gff_row["strand"]
	
	# Get upstream and downstream positions
	upstream_start = max(0, start - BP_considered - 1) # subtracting 1 for 0-based indexing (GFF is 1-based)
	downstream_end = start + BP_considered - 1
	
	# Extract sequence
	sequence = str(genome[seqid][upstream_start:downstream_end])
	
	# If on the negative strand, take the reverse complement
	if strand == "-":
		sequence = str(Seq(sequence).reverse_complement())
	
	return sequence

In [67]:
gene_sequences = {}
not_found_count = 0
for gene in tqdm(gene_names):
	gene_rows = genes[genes["attributes"].str.contains(f'ID=gene-{gene};')]
	if len(gene_rows) == 0:
		# print(f"No entry found for gene {gene}")
		not_found_count += 1
		continue
	elif len(gene_rows) > 1:
		print(f"Multiple entries found for gene {gene}, taking the first one.")
	gene_row = gene_rows.iloc[0]
	gene_sequence = get_target_gene_sequence(gene_row)
	gene_sequences[gene] = gene_sequence
print(f"Total genes not found: {not_found_count}")

100%|██████████| 5424/5424 [02:08<00:00, 42.14it/s]

Total genes not found: 117


### save dict to pkl

In [68]:
import pickle

In [69]:
with open(f"gene_sequences_{BP_considered*2}bp.pkl", "wb") as f:
    pickle.dump(gene_sequences, f)

In [70]:
gff = None; genes = None; genome = None
gene_sequences = None

## Process the tf's now

In [71]:
tf_names = list(adata.obs["target_gene"].unique())

In [72]:
len(tf_names)

223

## Read the rna seq file

In [73]:
from Bio import SeqIO

In [74]:
all_records = list(SeqIO.parse("GRCh38_latest_rna.gbff", "gb"))
tf_seqs = {}
for tf_name in tqdm(tf_names):
    for record in all_records:
        for feature in record.features:
            if feature.type == "gene":
                if ("gene" in feature.qualifiers and tf_name in feature.qualifiers["gene"]) or (
                    "gene_synonym" in feature.qualifiers and tf_name in feature.qualifiers["gene_synonym"]
                ):
                    start = feature.location.start
                    end = feature.location.end
                    new_seq_str = str(record.seq[start:end])  # ensure plain Python string
                    if tf_name not in tf_seqs:
                        tf_seqs[tf_name] = new_seq_str
                    else:
                        # take longer sequence if multiple found
                        if (end - start) > len(tf_seqs[tf_name]):
                            tf_seqs[tf_name] = new_seq_str

100%|██████████| 223/223 [01:27<00:00,  2.55it/s]


In [75]:
len(tf_names) - len(tf_seqs)

0

In [76]:
with open("tf_sequences.pkl", "wb") as f:
    pickle.dump(tf_seqs, f)

In [77]:
all_records = None
tf_seqs = None

## Create a dataframe, with each row having: tf_name, gene_name, gene_expression_change

In [78]:
df_records = []
for tf_name in tqdm(tf_names):
    tf_data = adata[adata.obs["target_gene"] == tf_name]
    expressions = tf_data.X.mean(axis=0)  # Convert to 1D array
    for gene_idx, gene_name in enumerate(gene_names):
        expression_value = expressions[gene_idx]
        df_records.append([tf_name, gene_name, expression_value])

100%|██████████| 223/223 [00:01<00:00, 117.33it/s]


In [79]:
df = pd.DataFrame(df_records, columns=["tf_name", "gene_name", "expression"])

In [80]:
df.shape

(1209552, 3)

In [81]:
len(tf_names) * len(gene_names)

1209552

In [82]:
df.head(3)

,tf_name,gene_name,expression
0,AATF,DPM1,-0.124948
1,AATF,CFH,0.016431
2,AATF,FUCA2,-0.104911


In [83]:
df.to_parquet("tf_gene_expression.parquet", index=False, engine='pyarrow')